# KPIs 

### Importo las librerias necesarias

In [53]:
import pandas as pd 
import numpy as np 
import herramientas 

Creo los dataframes para leer los archivos .csv

In [54]:
df_homicidios=pd.read_csv(r'data\homicidios_limpio.csv')
df_poblacion=pd.read_csv(r'data\poblacion_caba.csv')

Verifico la existencia de nulos y el tipo de dato

In [55]:
herramientas.verifica_tipo_y_nulos(df_homicidios)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,Id,[<class 'str'>],100.0,0.0,0
1,Cantidad de victimas,[<class 'int'>],100.0,0.0,0
2,Fecha,[<class 'str'>],100.0,0.0,0
3,Año,[<class 'int'>],100.0,0.0,0
4,Mes,[<class 'int'>],100.0,0.0,0
5,Dia,[<class 'int'>],100.0,0.0,0
6,Hora,[<class 'str'>],100.0,0.0,0
7,Hora en entero,[<class 'int'>],100.0,0.0,0
8,Lugar del hecho,[<class 'str'>],100.0,0.0,0
9,Tipo de calle,[<class 'str'>],100.0,0.0,0


In [56]:
herramientas.verifica_tipo_y_nulos(df_poblacion)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,Jurisdiccion,[<class 'str'>],100.0,0.0,0
1,2016,[<class 'int'>],100.0,0.0,0
2,2017,[<class 'int'>],100.0,0.0,0
3,2018,[<class 'int'>],100.0,0.0,0
4,2019,[<class 'int'>],100.0,0.0,0
5,2020,[<class 'int'>],100.0,0.0,0
6,2021,[<class 'int'>],100.0,0.0,0


### Ahora procedo a crear las tablas necesarias para cumplir con los kpis pedidos para luego importarlos en mi dashboard interactivo

- KPI 1: Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior

Inicialmente creo un dataframe para calcular la media de todos los años que necesito para mi analisis 

In [57]:
media_poblacion=df_poblacion[['2016', '2017', '2018', '2019','2020','2021']].mean(axis=1).astype(int)
media_poblacion

0    3069231
dtype: int32

Creo una clase para crear una columna Semestre, en df_homicidios

In [58]:
def determinar_semestre(mes):
    if mes <= 6:
        return 1
    else:
        return 2
df_homicidios["Semestre"] =df_homicidios["Mes"].apply(determinar_semestre)

Verifico la creacion de dicha columna 

In [59]:
df_homicidios.columns

Index(['Id', 'Cantidad de victimas', 'Fecha', 'Año', 'Mes', 'Dia', 'Hora',
       'Hora en entero', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Rol', 'Sexo', 'Edad',
       'Intervalo', 'HoraDia', 'DiaSemanaNum', 'DiaSemana', 'Dia de la semana',
       'Tipo de dia', 'Semestre'],
      dtype='object')

Agrupo los datos por las columnas "Año" y "Semestre", luego selecciono la columna "Cantidad de victimas" y suma los valores para cada grupo. Luego en la misma linea de codigo reinicio los indices del dataframe resultante, df_homicidios_p, y les doy el nombre de Victimas por Semestre Actual

In [60]:
df_homicidios_p=df_homicidios.groupby(["Año", "Semestre"])["Cantidad de victimas"].sum().reset_index(name='Victimas por Semestre actual')
df_homicidios_p

,Año,Semestre,Victimas por Semestre actual
0,2016,1,67
1,2016,2,83
2,2017,1,81
3,2017,2,79
4,2018,1,76
5,2018,2,85
6,2019,1,59
7,2019,2,47
8,2020,1,31
9,2020,2,56


Creo una nueva columna que muestra el número de víctimas del semestre anterior, utilizando los datos de la columna 'Victimas por Semestre actual' y desplazando esos valores una fila hacia arriba. Esto permite comparar las víctimas entre el semestre actual y el semestre anterior.

In [61]:
df_homicidios_p['Victimas por Semestre anterior'] = df_homicidios_p['Victimas por Semestre actual'].shift(periods=1, fill_value=0)
df_homicidios_p

,Año,Semestre,Victimas por Semestre actual,Victimas por Semestre anterior
0,2016,1,67,0
1,2016,2,83,67
2,2017,1,81,83
3,2017,2,79,81
4,2018,1,76,79
5,2018,2,85,76
6,2019,1,59,85
7,2019,2,47,59
8,2020,1,31,47
9,2020,2,56,31


Calculo las tasas de victimas actuales y anteriores por medio del siguiente calculo matematico: 
(Número de homicidios en siniestros viales / Población total) * 100,000 

In [62]:
df_homicidios_p['Tasa_Victimas_Actual'] = (df_homicidios_p['Victimas por Semestre actual']/3069231) * 100000
df_homicidios_p['Tasa_Victimas_Anterior'] = (df_homicidios_p['Victimas por Semestre anterior']/3069231) * 100000
df_homicidios_p

,Año,Semestre,Victimas por Semestre actual,Victimas por Semestre anterior,Tasa_Victimas_Actual,Tasa_Victimas_Anterior
0,2016,1,67,0,2.182957,0.000000
1,2016,2,83,67,2.704260,2.182957
2,2017,1,81,83,2.639098,2.704260
3,2017,2,79,81,2.573935,2.639098
4,2018,1,76,79,2.476190,2.573935
5,2018,2,85,76,2.769423,2.476190
6,2019,1,59,85,1.922306,2.769423
7,2019,2,47,59,1.531328,1.922306
8,2020,1,31,47,1.010025,1.531328
9,2020,2,56,31,1.824561,1.010025


Calculo la tasa de variacion

In [63]:
df_homicidios_p['Tasa_Variacion'] = (df_homicidios_p['Tasa_Victimas_Actual'] - df_homicidios_p['Tasa_Victimas_Anterior']) / df_homicidios_p['Tasa_Victimas_Anterior'] * 100
df_homicidios_p

,Año,Semestre,Victimas por Semestre actual,Victimas por Semestre anterior,Tasa_Victimas_Actual,Tasa_Victimas_Anterior,Tasa_Variacion
0,2016,1,67,0,2.182957,0.000000,inf
1,2016,2,83,67,2.704260,2.182957,23.880597
2,2017,1,81,83,2.639098,2.704260,-2.409639
3,2017,2,79,81,2.573935,2.639098,-2.469136
4,2018,1,76,79,2.476190,2.573935,-3.797468
5,2018,2,85,76,2.769423,2.476190,11.842105
6,2019,1,59,85,1.922306,2.769423,-30.588235
7,2019,2,47,59,1.531328,1.922306,-20.338983
8,2020,1,31,47,1.010025,1.531328,-34.042553
9,2020,2,56,31,1.824561,1.010025,80.645161


### Carga de la tabla kpi_1'

Se procede a guardar los cambios anteriormente realizados en un archivo llamado  **'kpi_1.csv'**

In [64]:
df_homicidios_p_limpio = 'data/kpi_1.csv'
df_homicidios_p.to_csv(df_homicidios_p_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {df_homicidios_p_limpio}')

Se guardó el archivo data/kpi_1.csv


- KPI 2: Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

Inicialmente traigo la columna Victima del df_homicidios

In [65]:
df_homicidios['Victima']

0           MOTO
1           AUTO
2           MOTO
3           MOTO
4           MOTO
         ...    
712         MOTO
713         MOTO
714         MOTO
715         AUTO
716    BICICLETA
Name: Victima, Length: 717, dtype: object

Creo un nuevo dataframe que en la columna Victimas solo me traiga los que digan Moto

In [66]:
df_homicidios_m=df_homicidios[df_homicidios['Victima']== 'MOTO']
df_homicidios_m.columns

Index(['Id', 'Cantidad de victimas', 'Fecha', 'Año', 'Mes', 'Dia', 'Hora',
       'Hora en entero', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Rol', 'Sexo', 'Edad',
       'Intervalo', 'HoraDia', 'DiaSemanaNum', 'DiaSemana', 'Dia de la semana',
       'Tipo de dia', 'Semestre'],
      dtype='object')

Agrupo los datos por las columnas "Año", luego selecciono la columna "Cantidad de victimas" y suma los valores para cada grupo. Luego en la misma linea de codigo reinicio los indices del dataframe resultante, df_homicidios_p, y les doy el nombre de Victimas por Año Actual

In [67]:
df_homicidios_m=df_homicidios_m.groupby(["Año"])["Cantidad de victimas"].size().reset_index(name='Victimas por Año actual')
df_homicidios_m

,Año,Victimas por Año actual
0,2016,65
1,2017,56
2,2018,57
3,2019,50
4,2020,28
5,2021,46


Creo una nueva columna que muestra el número de víctimas del año anterior, utilizando los datos de la columna 'Victimas por Año actual' y desplazando esos valores una fila hacia arriba. Esto permite comparar las víctimas entre el año actual y el año anterior.

In [68]:
df_homicidios_m['Victimas por Año anterior'] = df_homicidios_m['Victimas por Año actual'].shift(periods=1, fill_value=0)
df_homicidios_m

,Año,Victimas por Año actual,Victimas por Año anterior
0,2016,65,0
1,2017,56,65
2,2018,57,56
3,2019,50,57
4,2020,28,50
5,2021,46,28


Calculo la tasa de variacion

In [69]:
df_homicidios_m['Tasa_Variacion'] = (df_homicidios_m['Victimas por Año anterior'] - df_homicidios_m['Victimas por Año actual']) / df_homicidios_m['Victimas por Año anterior'] * 100

In [70]:
df_homicidios_m

,Año,Victimas por Año actual,Victimas por Año anterior,Tasa_Variacion
0,2016,65,0,-inf
1,2017,56,65,13.846154
2,2018,57,56,-1.785714
3,2019,50,57,12.280702
4,2020,28,50,44.000000
5,2021,46,28,-64.285714


### Carga de la tabla kpi_2'

Se procede a guardar los cambios anteriormente realizados en un archivo llamado  **'kpi_2.csv'**

In [71]:
df_homicidios_m_limpio = 'data/kpi_2.csv'
df_homicidios_m.to_csv(df_homicidios_m_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {df_homicidios_m_limpio}')

Se guardó el archivo data/kpi_2.csv


- KPI 3: Reducir en un 10% la cantidad de accidentes mortales de autos en el último año, en CABA, respecto al año anterior.

Inicialmente traigo la columna Victima del df_homicidios

In [72]:
df_homicidios['Victima']

0           MOTO
1           AUTO
2           MOTO
3           MOTO
4           MOTO
         ...    
712         MOTO
713         MOTO
714         MOTO
715         AUTO
716    BICICLETA
Name: Victima, Length: 717, dtype: object

Creo un nuevo dataframe que en la columna Victimas solo me traiga los que digan Auto

In [73]:
df_homicidios_a=df_homicidios[df_homicidios['Victima']== 'AUTO']
df_homicidios_a

,Id,Cantidad de victimas,Fecha,Año,Mes,Dia,Hora,Hora en entero,Lugar del hecho,Tipo de calle,...,Rol,Sexo,Edad,Intervalo,HoraDia,DiaSemanaNum,DiaSemana,Dia de la semana,Tipo de dia,Semestre
1,2016-0002,1,2016-01-02,2016,1,2,2023-12-05 01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,CONDUCTOR,MASCULINO,70,Madrugada,1,5,Sábado,5,Fin de Semana,1
15,2016-0021,1,2016-02-17,2016,2,17,2023-12-05 23:35:00,23,AV DIRECTORIO Y CURAPALIGUE,AVENIDA,...,CONDUCTOR,MASCULINO,34,Noche,23,2,Miércoles,2,Semana,1
17,2016-0025,1,2016-02-28,2016,2,28,2023-12-05 04:50:00,4,AV GRAL PAZ Y AV 27 DE FEBRERO,GRAL PAZ,...,PASAJERO_ACOMPAÑANTE,MASCULINO,34,Madrugada,4,6,Domingo,6,Fin de Semana,1
18,2016-0026,1,2016-02-28,2016,2,28,2023-12-05 07:30:00,7,HOLMBERG Y VEDIA,CALLE,...,CONDUCTOR,MASCULINO,30,Mañana,7,6,Domingo,6,Fin de Semana,1
19,2016-0027,1,2016-02-28,2016,2,28,2023-12-05 09:35:00,9,CURAPALIGUE Y AVELINO DIAZ,AVENIDA,...,PASAJERO_ACOMPAÑANTE,MASCULINO,34,Mañana,9,6,Domingo,6,Fin de Semana,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,2021-0059,1,2021-07-22,2021,7,22,2023-12-05 01:20:00,1,"GARAY, JUAN DE AV. 2269",AVENIDA,...,PASAJERO_ACOMPAÑANTE,FEMENINO,44,Madrugada,1,3,Jueves,3,Semana,2
681,2021-0062,1,2021-07-27,2021,7,27,2023-12-05 07:16:00,7,LA PLATA AV. 2723,AVENIDA,...,CONDUCTOR,MASCULINO,73,Mañana,7,1,Martes,1,Semana,2
682,2021-0063,1,2021-08-02,2021,8,2,2023-12-05 04:40:00,4,"ALBERDI, JUAN BAUTISTA AV. 2906",AVENIDA,...,PASAJERO_ACOMPAÑANTE,FEMENINO,31,Madrugada,4,0,Lunes,0,Semana,2
703,2021-0084,1,2021-11-11,2021,11,11,2023-12-05 03:00:00,3,TAPALQUE Y BRUIX AV.,AVENIDA,...,CONDUCTOR,MASCULINO,32,Madrugada,3,3,Jueves,3,Semana,2


Agrupo los datos por las columnas "Año", luego selecciono la columna "Cantidad de victimas" y suma los valores para cada grupo. Luego en la misma linea de codigo reinicio los indices del dataframe resultante, df_homicidios_p, y les doy el nombre de Victimas por Año Actual

In [74]:
df_homicidios_a=df_homicidios_a.groupby(["Año"])["Cantidad de victimas"].size().reset_index(name='Victimas por Año actual')
df_homicidios_a

,Año,Victimas por Año actual
0,2016,18
1,2017,23
2,2018,17
3,2019,11
4,2020,12
5,2021,11


Creo una nueva columna que muestra el número de víctimas del año anterior, utilizando los datos de la columna 'Victimas por Año actual' y desplazando esos valores una fila hacia arriba. Esto permite comparar las víctimas entre el año actual y el año anterior.

In [75]:
df_homicidios_a['Victimas por Año anterior'] = df_homicidios_a['Victimas por Año actual'].shift(periods=1, fill_value=0)
df_homicidios_a

,Año,Victimas por Año actual,Victimas por Año anterior
0,2016,18,0
1,2017,23,18
2,2018,17,23
3,2019,11,17
4,2020,12,11
5,2021,11,12


Calculo la tasa de variacion

In [76]:
df_homicidios_a['Tasa_Variacion'] = (df_homicidios_a['Victimas por Año anterior'] - df_homicidios_a['Victimas por Año actual']) / df_homicidios_a['Victimas por Año anterior'] * 100

In [77]:
df_homicidios_a

,Año,Victimas por Año actual,Victimas por Año anterior,Tasa_Variacion
0,2016,18,0,-inf
1,2017,23,18,-27.777778
2,2018,17,23,26.086957
3,2019,11,17,35.294118
4,2020,12,11,-9.090909
5,2021,11,12,8.333333


### Carga de la tabla kpi_3'

Se procede a guardar los cambios anteriormente realizados en un archivo llamado  **'kpi_3.csv'**

In [78]:
df_homicidios_a_limpio = 'data/kpi_3.csv'
df_homicidios_a.to_csv(df_homicidios_a_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {df_homicidios_a_limpio}')

Se guardó el archivo data/kpi_3.csv
